In [1]:
import pandas as pd 

#  - submition one was made with a randomforest classifier no params, scored in the 29th percentile, not great
#  - Second submition made with random forest tuned for the best hyperparams with a gridseach, better score, 
# - Thrid submition made with random forest, same tuning as for attempt 2 but with new features engineered

In [3]:
df = pd.read_csv('train.csv') 

In [4]:
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [40]:
X = df.drop(['Exited', 'Surname'], axis=1)
y = df['Exited'] 

testdf = pd.read_csv('test.csv') 
X_test = testdf.drop(['Surname'], axis=1)

In [64]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


# Defining categorical and numerical features
categorical_features = ['Geography', 'Gender']
numerical_features = X.drop(columns=categorical_features).columns.tolist()

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Creating a pipeline with preprocessing and classifier
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(n_estimators=200, min_samples_split=2, max_depth=10, random_state=42))])


# Training the model
model.fit(X, y)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['id', 'CustomerId',
                                                   'CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary',
                                                   'UtilizationRate',
                                                   'Age_NumOfProducts',
                                                   'CreditScore_IsActive',
                                                   'Age_Squared']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Geography', 'Gender'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, n_estimators=200,
                                        random_state=42))])

In [48]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X, y)

# Print the best parameters
print("Best parameters:", grid_search.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   8.3s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   8.3s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   8.5s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   8.4s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   8.4s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=200; total time=  16.8s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=200; total time=  17.3s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=200; total time=  17.1s
[C

In [2]:
# creating some test and train tests to test the effectivness of other aproaches like random forsts and XGB boost 

from sklearn.model_selection import train_test_split 

df = pd.read_csv('train.csv') 

# feature enginnering 
df['UtilizationRate'] = df['Balance'] / (df['NumOfProducts'] + 1)
df['Age_NumOfProducts'] = df['Age'] * df['NumOfProducts']
df['CreditScore_IsActive'] = df['CreditScore'] * df['IsActiveMember']
df['Age_Squared'] = df['Age'] ** 2


X = df.drop(['Exited', 'Surname'], axis=1)
y = df['Exited'] 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42) 
# making the new df the contest df for convienence with feature enginenering

df = pd.read_csv('test.csv') 

df['UtilizationRate'] = df['Balance'] / (df['NumOfProducts'] + 1)
df['Age_NumOfProducts'] = df['Age'] * df['NumOfProducts']
df['CreditScore_IsActive'] = df['CreditScore'] * df['IsActiveMember']
df['Age_Squared'] = df['Age'] ** 2

X_contest = df.drop(['Surname'], axis=1)


In [3]:
X.head()

,id,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,UtilizationRate,Age_NumOfProducts,CreditScore_IsActive,Age_Squared
0,0,15674932,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0.00,66.0,0.0,1089.0
1,1,15749177,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0.00,66.0,627.0,1089.0
2,2,15694510,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0.00,80.0,0.0,1600.0
3,3,15741417,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,74441.27,34.0,581.0,1156.0
4,4,15766172,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0.00,66.0,716.0,1089.0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Initialize XGBClassifier
xgb_model = XGBClassifier()

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])

# Hyperparameter grid
param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__max_depth': [2, 3, 5, 7],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.5]
}

# Use this pipeline in GridSearchCV with verbosity
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

# Now the best_model will be a pipeline
best_pipeline = grid_search.best_estimator_

# Fit the best model with early stopping (manually)
best_model = best_pipeline.named_steps['classifier']
best_model.fit(best_pipeline.named_steps['preprocessor'].transform(X_train), y_train, 
               early_stopping_rounds=50, eval_set=[(best_pipeline.named_steps['preprocessor'].transform(X_val), y_val)], 
   



In [55]:
# Ensure your ColumnTransformer is applied to the data
from sklearn.pipeline import Pipeline

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])

# Initialize XGBClassifier
xgb_model = XGBClassifier()

# Hyperparameter grid
param_grid = {
    'classifier__n_estimators': [350, 375, 400, 425, 450],
    'classifier__max_depth': [3, 4, 5, 6],
    'classifier__learning_rate': [0.05, 0.1, 0.15, 0.2],
    'classifier__min_child_weight': [1, 3, 5, 7],
    'classifier__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__gamma': [0, 0.1, 0.2, 0.3, 0.4]
}



# Use this pipeline in GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Now the best_model will be a pipeline
best_pipeline = grid_search.best_estimator_
best_model = best_pipeline.named_steps['classifier']

# Fit the best model with early stopping (using the pipeline)
best_pipeline.fit(X_train, y_train, classifier__early_stopping_rounds=50, classifier__eval_set=[(X_val, y_val)], classifier__verbose=True)

# Evaluate
y_pred = best_pipeline.predict(X_val)
print(classification_report(y_val, y_pred))


ValueError: Unknown model type

In [31]:
print(grid_search.best_params_)

{'classifier__subsample': 0.7, 'classifier__n_estimators': 420, 'classifier__min_child_weight': 5, 'classifier__max_depth': 6, 'classifier__learning_rate': 0.0125, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 0.8}


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np 

categorical_features = ['Geography', 'Gender']
numerical_features = X.drop(columns=categorical_features).columns

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Pipeline setup with a placeholder for the model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBClassifier())  # Default model
])

# Example parameter grid for RandomizedSearchCV
def model_picker(**params):
    model_type = params.pop('model__model_type')
    if model_type == 'xgb':
        # Filter out only xgb parameters
        xgb_params = {k.split("__")[2]: v for k, v in params.items() if 'xgb__' in k}
        return XGBClassifier(**xgb_params)
    elif model_type == 'lgbm':
        # Filter out only lgbm parameters
        lgbm_params = {k.split("__")[2]: v for k, v in params.items() if 'lgbm__' in k}
        return LGBMClassifier(**lgbm_params)
    else:
        raise ValueError("Unknown model type")


param_grid = {
    'model__model_type': ['xgb', 'lgbm'],
    'model__xgb__n_estimators': [100, 200, 300],
    'model__xgb__max_depth': [3, 6, 9],
    'model__xgb__learning_rate': [0.01, 0.1, 0.2],
    'model__xgb__subsample': [0.5, 0.7, 1.0],
    'model__xgb__colsample_bytree': [0.5, 0.7, 1.0],
    'model__lgbm__n_estimators': [100, 200, 300],
    'model__lgbm__max_depth': [3, 6, 9],
    'model__lgbm__learning_rate': [0.01, 0.1, 0.2],
    'model__lgbm__num_leaves': [31, 62, 127],
    'model__lgbm__feature_fraction': [0.5, 0.7, 1.0],
    'model__lgbm__bagging_fraction': [0.5, 0.7, 1.0],
    'model__lgbm__bagging_freq': [5, 7, 9]
}


# RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=50,
    scoring='accuracy',
    verbose=1,
    random_state=42,
    cv=5
)

# Fit the model
random_search.fit(X, y)

# After fitting, access the best parameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:43:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "lgbm__bagging_fraction", "lgbm__bagging_freq", "lgbm__feature_fraction", "lgbm__learning_rate", "lgbm__max_depth", "lgbm__n_estimators", "lgbm__num_leaves", "model_type", "xgb__colsample_bytree", "xgb__learning_rate", "xgb__max_depth", "xgb__n_estimators", "xgb__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:43:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "lgbm__bagging_fraction", "lgbm__bagging_freq", "lgbm__feature_fraction", "lgbm__learning_rate", "lgbm__max_depth", "lgbm__n_estimators", "lgbm__num_leaves", "model_type", "xgb__colsample_bytree", "xgb__learning_rate", "xgb__max_depth", "xgb__n_estimators", "xgb__s

KeyboardInterrupt: 

# Most promising code setup 

In [ ]:
# creating some test and train tests to test the effectivness of other aproaches like random forsts and XGB boost 

from sklearn.model_selection import train_test_split 

df = pd.read_csv('train.csv') 

# feature enginnering 
df['UtilizationRate'] = df['Balance'] / (df['NumOfProducts'])
df['Age_NumOfProducts'] = df['Age'] * df['NumOfProducts']
df['CreditScore_IsActive'] = df['CreditScore'] * df['IsActiveMember']
df['Age_Squared'] = df['Age'] ** 2


X = df.drop(['Exited', 'Surname'], axis=1)
y = df['Exited'] 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42) 
# making the new df the contest df for convienence with feature enginenering

df = pd.read_csv('test.csv') 

df['UtilizationRate'] = df['Balance'] / (df['NumOfProducts'])
df['Age_NumOfProducts'] = df['Age'] * df['NumOfProducts']
df['CreditScore_IsActive'] = df['CreditScore'] * df['IsActiveMember']
df['Age_Squared'] = df['Age'] ** 2

X_contest = df.drop(['Surname'], axis=1)

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np 

# Define your preprocessor
categorical_features = ['Geography', 'Gender']
numerical_features = X.drop(columns=categorical_features).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])


# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

param_grid = {
    'classifier__n_estimators': [350, 375, 400, 420, 425, 435],
    'classifier__max_depth': [2, 4, 5, 6, 7, 8],
    'classifier__learning_rate': [0.005, 0.01, 0.0125, 0.015, 0.0175, 0.0185, 0.02],
    'classifier__max_bin': [200, 256, 400, 512, 1024], 
    'classifier__min_child_weight': [1, 2, 3, 4, 5],
    'classifier__subsample': [0.6, 0.7, 0.75, 0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.6, 0.65, 0.7, 0.75, 0.8, 1.0],
    'classifier__gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'classifier__alpha': np.linspace(0.1, 1, 10),  
    'classifier__lambda': np.linspace(0.1, 1, 10)  
}

# Hyperparameter grid
# param_grid = {
#     'classifier__n_estimators': [200, 350, 375, 400, 420, 425, 435],
#     'classifier__max_depth': [2, 4, 5, 6, 7, 8],
#     'classifier__learning_rate': [0.005, 0.01, 0.0125, 0.015, 0.0175, 0.0185, 0.02],
#     'max_bin': [256, 512, 1024],  
#     'classifier__min_child_weight': [1, 2, 3, 4, 5],
#     'classifier__subsample': [0.6, 0.7, 0.75, 0.8, 0.9, 1.0],
#     'classifier__colsample_bytree': [0.6, 0.65, 0.7, 0.75, 0.8, 1.0],
#     'classifier__gamma': [0, 0.05, 0.1, 0.2, 0.3, 0.4]

# }


# Grid search with cross-validation
grid_search = RandomizedSearchCV(pipeline, param_grid, n_iter=50, cv=8, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X, y)

# Best model pipeline
best_pipeline = grid_search.best_estimator_

print("Best parameters:", grid_search.best_params_)

Fitting 8 folds for each of 50 candidates, totalling 400 fits
Best parameters: {'classifier__subsample': 0.9, 'classifier__n_estimators': 400, 'classifier__min_child_weight': 4, 'classifier__max_depth': 6, 'classifier__max_bin': 200, 'classifier__learning_rate': 0.01, 'classifier__lambda': 0.1, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 1.0, 'classifier__alpha': 0.7000000000000001}


In [7]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'classifier__subsample': 0.7, 'classifier__n_estimators': 350, 'classifier__min_child_weight': 5, 'classifier__max_depth': 7, 'classifier__max_bin': 256, 'classifier__learning_rate': 0.015, 'classifier__lambda': 0.1, 'classifier__gamma': 0.1, 'classifier__colsample_bytree': 0.75, 'classifier__alpha': 0.7000000000000001}


In [15]:
# Generating predictions for the entire dataset
predictions = best_pipeline.predict_proba(X_contest)[:, 1]

# Create a DataFrame for output
output = pd.DataFrame({'id': X_contest['id'], 'Exited': predictions})

print(output) 

            id    Exited
0       165034  0.022393
1       165035  0.816304
2       165036  0.033736
3       165037  0.245209
4       165038  0.345200
...        ...       ...
110018  275052  0.047670
110019  275053  0.102760
110020  275054  0.022450
110021  275055  0.155831
110022  275056  0.204409

[110023 rows x 2 columns]


In [16]:
output.to_csv('predictions.csv', index=False)

In [10]:
# Generating predictions for the entire dataset
# predictions = model.predict_proba(X_contest)[:, 1]

# # Create a DataFrame for output
# output = pd.DataFrame({'id': X_contest['id'], 'Exited': predictions})

# print(output) 